In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import regex as re
import pickle as pkl
import os

In [2]:
os.chdir("/Users/avascharfstein/Desktop/Dartmouth/mds-final")

In [3]:
df = pd.read_csv("data/df_cleaned.csv")
df.head()

,event_id,player_id_home,name_home,seed_home,aces_home,backhand_errors_home,backhand_unforced_errors_home,backhand_winners_home,breakpoints_won_home,double_faults_home,...,win_home,avg_set_length,avg_game_length,avg_points_per_game,match_length,rs_home,rs_away,avg_set_diff,avg_set_games,sets
0,sr:sport_event:42023445,sr:competitor:66968,"Kudermetova, Veronika",0,14,13,8,9,1,1,...,True,50.983333,4.433333,6.913043,101.966667,12.0,101.0,1.500000,11.500000,2
1,sr:sport_event:42023447,sr:competitor:41355,"Bogdan, Ana",1,6,12,7,7,3,2,...,True,72.141667,5.549359,6.769231,144.283333,61.0,15.0,1.000000,13.000000,2
2,sr:sport_event:42023449,sr:competitor:256547,"Parks, Alycia",1,5,8,17,3,4,5,...,True,50.866667,5.354386,7.210526,101.733333,47.0,87.0,2.500000,9.500000,2
3,sr:sport_event:42023459,sr:competitor:99131,"Bucsa, Cristina",1,6,34,15,10,6,4,...,True,54.772222,5.134896,7.187500,164.316667,77.0,70.0,0.666667,10.666667,3
4,sr:sport_event:42023461,sr:competitor:44834,"Pegula, Jessica",0,3,20,14,11,5,3,...,True,46.783333,4.678333,6.600000,140.350000,4.0,44.0,2.000000,10.000000,3


Combining similar variables (winners and errors) ..?

In [4]:
errors = []
unforced_errors = []
winners = []
for col in df.columns:
    if "errors" in col:
        er = re.match(r"^(.+_errors)(_[a-z]+)$", col).group(1) 
        if "unforced" in col:
            if er not in unforced_errors:
                unforced_errors.append(er)
        else:
            if er not in errors:
                errors.append(er)
    elif "winners" in col:
        win = re.match(r"^(.+_winners)(_[a-z]+)$", col).group(1) 
        if win not in winners:
            winners.append(win)

print(errors)
print(unforced_errors)
print(winners)


['backhand_errors', 'forehand_errors', 'groundstroke_errors', 'overhead_stroke_errors', 'return_errors']
['backhand_unforced_errors', 'drop_shot_unforced_errors', 'forehand_unforced_errors', 'groundstroke_unforced_errors', 'lob_unforced_errors', 'overhead_stroke_unforced_errors', 'volley_unforced_errors']
['backhand_winners', 'drop_shot_winners', 'forehand_winners', 'groundstroke_winners', 'lob_winners', 'overhead_stroke_winners', 'return_winners', 'volley_winners']


In [5]:
# # add double faults to errors
# errors.append("double_faults")

In [6]:
for who in ['_home','_away']:
    df["errors"+who] = df[[err+who for err in errors]].sum(axis=1)
    df["unforced_errors"+who] = df[[err+who for err in unforced_errors]].sum(axis=1)
    df['winners'+who] = df[[win+who for win in winners]].sum(axis=1)
    df = df.drop(columns = [err+who for err in errors]+[err+who for err in unforced_errors]+[win+who for win in winners])

In [7]:
df.head()

,event_id,player_id_home,name_home,seed_home,aces_home,breakpoints_won_home,double_faults_home,first_serve_points_won_home,first_serve_successful_home,games_won_home,...,rs_away,avg_set_diff,avg_set_games,sets,errors_home,unforced_errors_home,winners_home,errors_away,unforced_errors_away,winners_away
0,sr:sport_event:42023445,sr:competitor:66968,"Kudermetova, Veronika",0,14,1,1,41,51,13.0,...,101.0,1.500000,11.500000,2,62,48,58,52,26,32
1,sr:sport_event:42023447,sr:competitor:41355,"Bogdan, Ana",1,6,3,2,36,66,14.0,...,15.0,1.000000,13.000000,2,60,35,26,51,82,54
2,sr:sport_event:42023449,sr:competitor:256547,"Parks, Alycia",1,5,4,5,21,30,12.0,...,87.0,2.500000,9.500000,2,28,54,40,40,52,24
3,sr:sport_event:42023459,sr:competitor:99131,"Bucsa, Cristina",1,6,6,4,48,76,17.0,...,70.0,0.666667,10.666667,3,100,66,71,85,50,42
4,sr:sport_event:42023461,sr:competitor:44834,"Pegula, Jessica",0,3,5,3,42,59,18.0,...,44.0,2.000000,10.000000,3,71,56,36,89,70,37


In [8]:
sorted(df.columns)

['aces_away',
 'aces_home',
 'avg_game_length',
 'avg_points_per_game',
 'avg_set_diff',
 'avg_set_games',
 'avg_set_length',
 'best_of',
 'breakpoints_won_away',
 'breakpoints_won_home',
 'competition',
 'date',
 'double_faults_away',
 'double_faults_home',
 'errors_away',
 'errors_home',
 'event_id',
 'first_serve_points_won_away',
 'first_serve_points_won_home',
 'first_serve_successful_away',
 'first_serve_successful_home',
 'games_won_away',
 'games_won_home',
 'match_length',
 'max_games_in_a_row_away',
 'max_games_in_a_row_home',
 'max_points_in_a_row_away',
 'max_points_in_a_row_home',
 'men',
 'name_away',
 'name_home',
 'player_id_away',
 'player_id_home',
 'points_won_away',
 'points_won_from_last_10_away',
 'points_won_from_last_10_home',
 'points_won_home',
 'round',
 'rs_away',
 'rs_home',
 'second_serve_points_won_away',
 'second_serve_points_won_home',
 'second_serve_successful_away',
 'second_serve_successful_home',
 'seed_away',
 'seed_home',
 'service_games_won_away'

In [9]:
context_vars = ["event_id","date","name_home","name_away","competition","player_id_home","player_id_away", "best_of","round"]
context = df[context_vars]
with open('data/context_vars.pkl', 'wb') as f:
    pkl.dump(context_vars, f)

In [10]:
stats_df = df.drop(columns = context_vars)
stats_df.insert(0, "men", stats_df.pop("men"))
stats_df

,men,seed_home,aces_home,breakpoints_won_home,double_faults_home,first_serve_points_won_home,first_serve_successful_home,games_won_home,max_games_in_a_row_home,max_points_in_a_row_home,...,rs_away,avg_set_diff,avg_set_games,sets,errors_home,unforced_errors_home,winners_home,errors_away,unforced_errors_away,winners_away
0,False,0,14,1,1,41,51,13.0,3.0,5,...,101.0,1.500000,11.500000,2,62,48,58,52,26,32
1,False,1,6,3,2,36,66,14.0,2.0,6,...,15.0,1.000000,13.000000,2,60,35,26,51,82,54
2,False,1,5,4,5,21,30,12.0,3.0,7,...,87.0,2.500000,9.500000,2,28,54,40,40,52,24
3,False,1,6,6,4,48,76,17.0,3.0,9,...,70.0,0.666667,10.666667,3,100,66,71,85,50,42
4,False,0,3,5,3,42,59,18.0,4.0,6,...,44.0,2.000000,10.000000,3,71,56,36,89,70,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,True,0,8,9,9,41,55,18.0,6.0,6,...,8.0,2.333333,9.666667,3,36,63,36,77,62,51
941,True,0,3,4,3,42,55,18.0,3.0,8,...,12.0,3.000000,9.000000,3,38,60,52,67,61,24
942,True,1,5,2,5,43,72,11.0,3.0,5,...,2.0,-2.666667,10.000000,3,79,72,43,67,42,47
943,True,0,0,1,0,64,93,16.0,3.0,6,...,3.0,-1.500000,9.500000,4,124,76,88,83,46,43


In [11]:
#stats_df['avg_set_diff'] = abs(stats_df['avg_set_diff'])

In [12]:
overall_stats = []
specific_stats = set()
specific_stats_all = []
for col in stats_df.columns:
    if "home" not in col and "away" not in col:
        overall_stats.append(col)
    elif "home" in col:
        specific_stats.add(re.match(r"^(.*?)\_home",col).group(1))
        specific_stats_all.append(col)
    elif "away" in col:
        specific_stats.add(re.match(r"^(.*?)\_away",col).group(1))
        specific_stats_all.append(col)

In [13]:
with open('data/specific_stats.pkl', 'wb') as f:
    pkl.dump(specific_stats, f)

with open('data/overall_stats.pkl', 'wb') as f:
    pkl.dump(overall_stats, f)

In [14]:
specific_stats

{'aces',
 'breakpoints_won',
 'double_faults',
 'errors',
 'first_serve_points_won',
 'first_serve_successful',
 'games_won',
 'max_games_in_a_row',
 'max_points_in_a_row',
 'points_won',
 'points_won_from_last_10',
 'rs',
 'second_serve_points_won',
 'second_serve_successful',
 'seed',
 'service_games_won',
 'service_points_lost',
 'service_points_won',
 'tiebreaks_won',
 'total_breakpoints',
 'unforced_errors',
 'win',
 'winners'}

In [15]:
overall_stats
# Only match_length from this list needs to be 'normalized'

['men',
 'avg_set_length',
 'avg_game_length',
 'avg_points_per_game',
 'match_length',
 'avg_set_diff',
 'avg_set_games',
 'sets']

New feature?

In [16]:
# df['expected_win'] = (df['win'+h] & df['rs_diff']>0) | (~df['win'+h] & df['rs_diff']<0)

In [17]:
h = '_home'
a = '_away'

def both(col):
    return [col+h,col+a]

In [18]:
from itertools import chain
# Normalize by dividing by the number of sets played in a given match
to_normalize = list(specific_stats_all.copy())
not_to = both('rs')+both('max_games_in_a_row')+both('max_points_in_a_row')+both('seed')+both('win')
to_normalize = [x for x in to_normalize if x not in not_to]
to_normalize.append('match_length')
stats_df[to_normalize] = stats_df[to_normalize].div(df['sets'], axis=0)

In [19]:
stats_df

,men,seed_home,aces_home,breakpoints_won_home,double_faults_home,first_serve_points_won_home,first_serve_successful_home,games_won_home,max_games_in_a_row_home,max_points_in_a_row_home,...,rs_away,avg_set_diff,avg_set_games,sets,errors_home,unforced_errors_home,winners_home,errors_away,unforced_errors_away,winners_away
0,False,0,7.000000,0.500000,0.500000,20.500000,25.500000,6.500000,3.0,5,...,101.0,1.500000,11.500000,2,31.000000,24.000000,29.000000,26.000000,13.000000,16.000000
1,False,1,3.000000,1.500000,1.000000,18.000000,33.000000,7.000000,2.0,6,...,15.0,1.000000,13.000000,2,30.000000,17.500000,13.000000,25.500000,41.000000,27.000000
2,False,1,2.500000,2.000000,2.500000,10.500000,15.000000,6.000000,3.0,7,...,87.0,2.500000,9.500000,2,14.000000,27.000000,20.000000,20.000000,26.000000,12.000000
3,False,1,2.000000,2.000000,1.333333,16.000000,25.333333,5.666667,3.0,9,...,70.0,0.666667,10.666667,3,33.333333,22.000000,23.666667,28.333333,16.666667,14.000000
4,False,0,1.000000,1.666667,1.000000,14.000000,19.666667,6.000000,4.0,6,...,44.0,2.000000,10.000000,3,23.666667,18.666667,12.000000,29.666667,23.333333,12.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,True,0,2.666667,3.000000,3.000000,13.666667,18.333333,6.000000,6.0,6,...,8.0,2.333333,9.666667,3,12.000000,21.000000,12.000000,25.666667,20.666667,17.000000
941,True,0,1.000000,1.333333,1.000000,14.000000,18.333333,6.000000,3.0,8,...,12.0,3.000000,9.000000,3,12.666667,20.000000,17.333333,22.333333,20.333333,8.000000
942,True,1,1.666667,0.666667,1.666667,14.333333,24.000000,3.666667,3.0,5,...,2.0,-2.666667,10.000000,3,26.333333,24.000000,14.333333,22.333333,14.000000,15.666667
943,True,0,0.000000,0.250000,0.000000,16.000000,23.250000,4.000000,3.0,6,...,3.0,-1.500000,9.500000,4,31.000000,19.000000,22.000000,20.750000,11.500000,10.750000


In [20]:
stats_df.to_csv('data/df_engineered.csv', index=False)